In [1]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

def split_message_to_bytes(message, num_blocks):
    words = message.split()
    words_per_block = len(words) // num_blocks
    blocks = []
    remainder = len(words) % num_blocks
    
    start = 0
    for i in range(num_blocks):
        if remainder > 0:
            block_length = words_per_block + 1
            remainder -= 1
        else:
            block_length = words_per_block
        
        block = ' '.join(words[start:start+block_length])
        block_bytes = block.encode('utf-8')
        blocks.append(block_bytes)
        
        start += block_length

    return blocks

blocks = []
m = "This message is for practicing AONT distributed processing."
s = 5

blocks_bytes = split_message_to_bytes(m, s)

for i, block_bytes in enumerate(blocks_bytes):
    print(f"m_{i+1}: {block_bytes}")

key_length = 16
k = os.urandom(key_length)
print("共通鍵暗号の鍵:", k)

key_length = 16
k_0 = os.urandom(key_length)
print("公開された共通鍵暗号の鍵:", k_0)

def pad(data):
    padding_length = 16 - (len(data) % 16)
    padded_data = data + bytes([padding_length] * padding_length)
    return padded_data

def Enc(key, plaintext):
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    padded_plaintext = pad(plaintext)

    ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()
    
    return iv + ciphertext

m2_blocks = []
for i, block in enumerate(blocks_bytes, start=1):
    block_bytes = block
    
    encrypted_block = Enc(k, bytes([i]))
    m2_block = bytes(a ^ b for a, b in zip(block_bytes, encrypted_block))
    
    m2_blocks.append(m2_block)

for i, m2_block in enumerate(m2_blocks, start=1):
    print(f"m2_{i}: {m2_block}")

hash_values = []
for i, block in enumerate(blocks_bytes, start=1):
    block_bytes = block
    xor_result = bytes(a ^ b for a, b in zip(block_bytes, bytes([i])))
    hash_value = Enc(k_0, xor_result)
    hash_values.append(hash_value)

for i, hash_value in enumerate(hash_values, start=1):
    print(f"h_{i}: {hash_value}")

m2_splus1 = k
for hash_value in hash_values:
    m2_splus1 = bytes(a ^ b for a, b in zip(m2_splus1, hash_value))

print(f"m2_(s+1): {m2_splus1}")

for i, hash_value in enumerate(hash_values, start=1):
    print(f"h_{i}: {hash_value}")

def xor_bytes(bytes_list):
    result = bytes_list[0]
    for b in bytes_list[1:]:
        result = bytes(a ^ b for a, b in zip(result, b))
    return result

result = xor_bytes([m2_splus1] + hash_values)
print("復号された共通鍵:", result)

# 共通鍵が一致しているかどうかを判定
if k == result:
    print("共通鍵が一致しています。")
else:
    print("共通鍵が一致していません。")

decrypted_blocks = []
for i, m2_block in enumerate(m2_blocks, start=1):
    decrypted_block= bytes(a ^ b for a, b in zip(m2_block, result))   
    print(f"復号されたm_{i}: {decrypted_block}")
    decrypted_blocks.append(decrypted_block)

decrypted_message = b"".join(decrypted_block for decrypted_block in decrypted_blocks)
print("復号されたメッセージ:", decrypted_message)

m_1: b'This message'
m_2: b'is for'
m_3: b'practicing AONT'
m_4: b'distributed'
m_5: b'processing.'
共通鍵暗号の鍵: b'\x1c\xb6\xb1\xf4\xe9\x9e\xbeZ=\x0b*C\xe3\x00\xfa^'
公開された共通鍵暗号の鍵: b'\xa9\x86\xab\xa40C\xc8\x19\xaa%\x06#\xdfa;X'
m2_1: b"\xd2\xabe\xa8*$:<\xba\xba'\x8a"
m2_2: b'\xf3L]zj6'
m2_3: b'\x86\xfa\xca\x9f\x14Q\xe6\x9f,\x03\\\xf8xqs'
m2_4: b'\xf76\xba\xac\x80@\xe3\xe3\n\xf1\x0c'
m2_5: b'\x95\xbek\xa0N\xa3\xa9\x0b\xf8B\x00'
h_1: b'\x7f\x9ac\xc4\xff\xb5`\xb8\x1d\x0eL\xbb\xe5\xc4?K\x92\x9c\xb2&O\x97\xf2\xbb\x00d9L\t\x1f\x16\x16'
h_2: b'\xc4=\x87\xd2\xce\xe5\xbc|"\xafx\x06;\x95\x82@M\x11\x92\xa1u\x81\x93\x15-\x93R\xdc7$6\xdc'
h_3: b'\xe6\xc7?\xae\xcb;b\xe7&\xf2\x18\x1d\x8d 6\xf2cV\x0b\xac\xbb\xba{%\x03\xe3">\xac"\xf5\xfe'
h_4: b'\x8ez\xcf\xa7\xab/\x84\x9d\x9e\xd4\xd4W\x01\xea\xf6?\xe8\x9e\xee;\xf0\x1cZS\xec\xe4b\x13\xf7\xd1\x81:'
h_5: b'\xfe\x1b\xdc\xad\xb1`\xce\x9e\x9d5\xd2L\xf3\xa4\x1d\xa8\x9be\x05\xe6O*\xf7\x167\xab\xef\xc4\x00 \xccb'
m2_(s+1): b"1\xb7yF\t\xbaJz'\xb9\x00\xf8B?\x9a0"
h_1: